In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [202]:
game_request = requests.get('https://www.basketball-reference.com/boxscores/pbp/201610260NOP.html')

In [203]:
game_request

<Response [200]>

In [204]:
game_soup = BeautifulSoup(game_request.text, 'lxml')

In [205]:
game_soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202007272" rel="dns-prefetch"/>
<!-- no:cookie fast load the css.           -->
<script>function gup(n) {n = n.replace(/[\[]/, '\\[').replace(/[\]]/, '\\]'); var r = new RegExp('[\\?&]'+n+'=([^&#]*)'); var re = r.exec(location.search);   return re === null?'':decodeURIComponent(re[1].replace(/\+/g,' '));}; document.srdev = gup('srdev')</script>
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="https://www.google-analytics.com" rel="preconnect"/>
<link crossorigin="" href="https://www.googletagservices.com" rel="preconnect"/>
<link as="script" crossorig

In [206]:
play_by_play = game_soup.find_all('tr')
play_by_play

[<tr class="thead" id="q1">
 <th colspan="6">1st Q</th>
 </tr>, <tr class="thead">
 <th aria-label="Time" class="center" data-stat="Time">Time</th>
 <th aria-label="Denver" class="center" data-stat="Denver">Denver</th>
 <th aria-label="" class="center" data-stat=""></th>
 <th aria-label="Score" class="center" data-stat="Score">Score</th>
 <th aria-label="" class="center" data-stat=""></th>
 <th aria-label="New Orleans" class="center" data-stat="New Orleans">New Orleans</th>
 </tr>, <tr>
 <td>12:00.0</td>
 <td class="center" colspan="5">Start of 1st quarter</td>
 </tr>, <tr>
 <td>12:00.0</td>
 <td class="center" colspan="5">Jump ball: <a href="/players/d/davisan02.html">A. Davis</a> vs. <a href="/players/n/nurkiju01.html">J. Nurkić</a> (<a href="/players/m/mudiaem01.html">E. Mudiay</a> gains possession)</td>
 </tr>, <tr>
 <td>11:39.0</td>
 <td class="">Turnover by <a href="/players/j/jokicni01.html">N. Jokić</a> (bad pass; steal by <a href="/players/d/davisan02.html">A. Davis</a>)</td><

In [207]:
len(play_by_play[3].contents)

5

In [208]:
len(play_by_play[5].contents)

9

In [209]:
for val in play_by_play:
    if len(val.contents) < 9:
        print(val)

<tr class="thead" id="q1">
<th colspan="6">1st Q</th>
</tr>
<tr>
<td>12:00.0</td>
<td class="center" colspan="5">Start of 1st quarter</td>
</tr>
<tr>
<td>12:00.0</td>
<td class="center" colspan="5">Jump ball: <a href="/players/d/davisan02.html">A. Davis</a> vs. <a href="/players/n/nurkiju01.html">J. Nurkić</a> (<a href="/players/m/mudiaem01.html">E. Mudiay</a> gains possession)</td>
</tr>
<tr>
<td>0:00.0</td>
<td class="center" colspan="5">End of 1st quarter</td>
</tr>
<tr class="thead" id="q2">
<th colspan="6">2nd Q</th>
</tr>
<tr>
<td>12:00.0</td>
<td class="center" colspan="5">Start of 2nd quarter</td>
</tr>
<tr>
<td>0:00.0</td>
<td class="center" colspan="5">End of 2nd quarter</td>
</tr>
<tr class="thead" id="q3">
<th colspan="6">3rd Q</th>
</tr>
<tr>
<td>12:00.0</td>
<td class="center" colspan="5">Start of 3rd quarter</td>
</tr>
<tr>
<td>0:36.0</td>
<td class="center" colspan="5">Jump ball: <a href="/players/f/farieke01.html">K. Faried</a> vs. <a href="/players/j/joneste01.html">T

In [210]:
play_by_play[10].contents[7]

<td class="bbr-play-score"><a href="/players/m/mooreet01.html">E. Moore</a> makes 2-pt jump shot from 8 ft (assist by <a href="/players/d/davisan02.html">A. Davis</a>)</td>

In [212]:
# Creating function for finding main player / secondary player

def main_secondary_players(content_val):
    players = content_val.find_all()
    
    if len(players) == 0:
        return None, None
    
    # If there's just one player in focus
    elif len(players) == 1:
        return players[0].attrs['href'], None
    
    else:
        return players[0].attrs['href'], players[1].attrs['href']

In [213]:
content_val = play_by_play[5].contents[3]
print(content_val)
players = content_val.find_all()

main_secondary_players(content_val)

<td> </td>


(None, None)

In [219]:
all_data = []

for line in play_by_play:
    content_information = line.contents
    if len(content_information) == 3:
        period = content_information[1].text
        #print(period)
        continue
    elif len(content_information) == 9:
        time_remaining = content_information[1].text.strip()
        
        visiting_text = content_information[3].text.strip()
        
        visiting_points = content_information[4].text.strip()
        
        visiting_score = content_information[5].text.split('-')[0]
        home_score = content_information[5].text.split('-')[1]
        
        home_points = content_information[6].text.strip()
        home_text = content_information[7].text.strip()
        
        
        if home_text != '':
            main_player, secondary_player = main_secondary_players(content_information[7])
        
        else:
            main_player, secondary_player = main_secondary_players(content_information[3])
        
        
        data = [period, time_remaining, visiting_text, visiting_points, visiting_score, 
                home_score, home_points, home_text,
                main_player, secondary_player]
        
        all_data.append(data)

In [220]:
df_columns = ['Period','Time','Visiting Posession','Visiting Points','Visiting Score',
              'Home Score','Home Points','Home Posession',
              'Main Player','Secondary Player']
df = pd.DataFrame(all_data, columns = df_columns)
df

,Period,Time,Visiting Posession,Visiting Points,Visiting Score,Home Score,Home Points,Home Posession,Main Player,Secondary Player
0,1st Q,11:39.0,Turnover by N. Jokić (bad pass; steal by A. Da...,,0,0,,,/players/j/jokicni01.html,/players/d/davisan02.html
1,1st Q,11:35.0,,,0,0,,Turnover by S. Hill (bad pass; steal by E. Mud...,/players/h/hillso01.html,/players/m/mudiaem01.html
2,1st Q,11:28.0,Shooting foul by Ö. Aşık (drawn by J. Nurkić),,0,0,,,/players/a/asikom01.html,/players/n/nurkiju01.html
3,1st Q,11:28.0,J. Nurkić misses free throw 1 of 2,,0,0,,,/players/n/nurkiju01.html,None
4,1st Q,11:28.0,Offensive rebound by Team,,0,0,,,None,None
...,...,...,...,...,...,...,...,...,...,...
491,4th Q,0:08.0,Defensive rebound by E. Mudiay,,105,100,,,/players/m/mudiaem01.html,None
492,4th Q,0:08.0,Personal take foul by E. Moore (drawn by E. Mu...,,105,100,,,/players/m/mooreet01.html,/players/m/mudiaem01.html
493,4th Q,0:08.0,E. Mudiay makes free throw 1 of 2,+1,106,100,,,/players/m/mudiaem01.html,None
494,4th Q,0:08.0,E. Mudiay makes free throw 2 of 2,+1,107,100,,,/players/m/mudiaem01.html,None


In [223]:
# Home Action
home_team_data = df[df['Home Posession'] != ''][['Period','Time','Home Posession','Home Points',
                                    'Home Score','Main Player','Secondary Player']]

# Visiting Action
visiting_team_data = df[df['Visiting Posession'] != ''][['Period','Time','Visiting Posession','Visiting Points',
                                    'Visiting Score','Main Player','Secondary Player']]

In [231]:
full_columns = ['Period','Time','Posession','Points','Score','Main Player','Secondary Player','Team']

In [232]:
home_team_data['Team'] = 'Home'
home_team_data.columns = full_columns
home_team_data.head()

,Period,Time,Posession,Points,Score,Main Player,Secondary Player,Team
1,1st Q,11:35.0,Turnover by S. Hill (bad pass; steal by E. Mud...,,0,/players/h/hillso01.html,/players/m/mudiaem01.html,Home
6,1st Q,11:11.0,E. Moore makes 2-pt jump shot from 8 ft (assis...,+2,2,/players/m/mooreet01.html,/players/d/davisan02.html,Home
8,1st Q,10:41.0,A. Davis misses 2-pt jump shot from 21 ft,,2,/players/d/davisan02.html,None,Home
11,1st Q,10:32.0,Defensive rebound by Team,,2,None,None,Home
13,1st Q,10:11.0,Turnover by E. Moore (bad pass; steal by D. Ga...,,2,/players/m/mooreet01.html,/players/g/gallida01.html,Home


In [233]:
visiting_team_data['Team'] = 'Away'
visiting_team_data.columns = full_columns
visiting_team_data.head()

,Period,Time,Posession,Points,Score,Main Player,Secondary Player,Team
0,1st Q,11:39.0,Turnover by N. Jokić (bad pass; steal by A. Da...,,0,/players/j/jokicni01.html,/players/d/davisan02.html,Away
2,1st Q,11:28.0,Shooting foul by Ö. Aşık (drawn by J. Nurkić),,0,/players/a/asikom01.html,/players/n/nurkiju01.html,Away
3,1st Q,11:28.0,J. Nurkić misses free throw 1 of 2,,0,/players/n/nurkiju01.html,None,Away
4,1st Q,11:28.0,Offensive rebound by Team,,0,None,None,Away
5,1st Q,11:28.0,J. Nurkić makes free throw 2 of 2,+1,1,/players/n/nurkiju01.html,None,Away


In [235]:
home_team_data.append(visiting_team_data).sort_index()

,Period,Time,Posession,Points,Score,Main Player,Secondary Player,Team
0,1st Q,11:39.0,Turnover by N. Jokić (bad pass; steal by A. Da...,,0,/players/j/jokicni01.html,/players/d/davisan02.html,Away
1,1st Q,11:35.0,Turnover by S. Hill (bad pass; steal by E. Mud...,,0,/players/h/hillso01.html,/players/m/mudiaem01.html,Home
2,1st Q,11:28.0,Shooting foul by Ö. Aşık (drawn by J. Nurkić),,0,/players/a/asikom01.html,/players/n/nurkiju01.html,Away
3,1st Q,11:28.0,J. Nurkić misses free throw 1 of 2,,0,/players/n/nurkiju01.html,None,Away
4,1st Q,11:28.0,Offensive rebound by Team,,0,None,None,Away
...,...,...,...,...,...,...,...,...
491,4th Q,0:08.0,Defensive rebound by E. Mudiay,,105,/players/m/mudiaem01.html,None,Away
492,4th Q,0:08.0,Personal take foul by E. Moore (drawn by E. Mu...,,105,/players/m/mooreet01.html,/players/m/mudiaem01.html,Away
493,4th Q,0:08.0,E. Mudiay makes free throw 1 of 2,+1,106,/players/m/mudiaem01.html,None,Away
494,4th Q,0:08.0,E. Mudiay makes free throw 2 of 2,+1,107,/players/m/mudiaem01.html,None,Away
